<a href="https://colab.research.google.com/github/SVJLucas/NeuralSchrodinger-Solving-the-Schrodinger-Equation-using-Neural-Networks/blob/main/Equation_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Equation Engineering

\begin{align}
i ℏ
\frac{∂Ψ(t,r,θ,φ)}{∂t}=V(t,r,θ,φ)\cdot Ψ(t,r,θ,φ) -\frac{ℏ^2}{2m} ∇^2 Ψ(t,r,θ,φ)
\end{align}

The Schrodinger equation, as it is given, is not properly suited for use in a neural network. This is because the equation involves complex numbers, which can be challenging to work with in a neural network. Additionally, there is the question of which system of units to use for the various constants and other arguments in the equation. All of these factors make it difficult to use the Schrodinger equation as it is in a neural network. So, first, let's do *Equation Engineering*, to make it more like a output of a neural network.

### Writing the equation

We will be using the Python library Sympy to write the Schrodinger equation for the hydrogen atom. However, before we do that, we need to define the necessary physical constants, such as the electron mass and the elementary charge, and we also need to write a function that calculates the Laplacian in spherical coordinates, which is used in the Schrodinger equation. This will allow us to write the equation in a way that is compatible with numerical methods, such as those used in neural networks.

In [1]:
import sympy as sp

Defining the constants:

In [132]:
i = sp.I
t,r,θ,φ,μ,ħ,e,ɛ_0 = sp.symbols('t r θ φ μ ℏ e ɛ_0',real=True,positive=True)
Ψ,Ψ_re,Ψ_im,V,Δ = sp.symbols('Ψ Ψ_re Ψ_im V Δ',cls=sp.Function,real=True)
Ψ = Ψ(t,r,θ,φ)
Ψ_re = Ψ_re(t,r,θ,φ)
Ψ_im = Ψ_im(t,r,θ,φ)
V = V(t,r,θ,φ)

Defining the Laplacian:

In [159]:
def Δ(r, θ, φ, Ψ):

    """
    Calculates the Laplacian operator (∇²) on the function Ψ, given the spherical coordinates r, θ, and φ.

    Parameters:
        r (sympy.core.symbol.Symbol): The radial coordinate in spherical coordinates.
        θ (sympy.core.symbol.Symbol): The polar angle coordinate in spherical coordinates.
        φ (sympy.core.symbol.Symbol): The azimuthal angle coordinate in spherical coordinates.
        Ψ (sympy.core.function.Function): The function on which to operate the Laplacian operator.

    Returns:
        sympy.core.expr.Expr: The result of applying the Laplacian operator to Ψ.

    """

    # Calculate the first term of the Laplacian operator
    r_term = (1 / r**2) * sp.Derivative((r**2) * sp.Derivative(Ψ, r), r)

    # Calculate the second term of the Laplacian operator
    θ_term = (1 / ((r**2) * sp.sin(θ))) * sp.Derivative(sp.sin(θ) * sp.Derivative(Ψ, θ), θ)

    # Calculate the third term of the Laplacian operator
    φ_term = (1 / ((r**2) * sp.sin(θ)**2)) * sp.Derivative(Ψ, (φ, 2))

    # Return the sum of all three terms
    return r_term + θ_term + φ_term

Now we can write the Schrödinger's Equation:

In [160]:
schrodinger = sp.Eq(i*ħ*sp.Derivative(Ψ,t),(-((ħ**2)/(2*μ)*Δ(r,θ,φ,Ψ))+V*Ψ).expand())
schrodinger

Eq(I*ℏ*Derivative(Ψ(t, r, θ, φ), t), V(t, r, θ, φ)*Ψ(t, r, θ, φ) - ℏ**2*Derivative(r**2*Derivative(Ψ(t, r, θ, φ), r), r)/(2*r**2*μ) - ℏ**2*Derivative(sin(θ)*Derivative(Ψ(t, r, θ, φ), θ), θ)/(2*r**2*μ*sin(θ)) - ℏ**2*Derivative(Ψ(t, r, θ, φ), (φ, 2))/(2*r**2*μ*sin(θ)**2))

### The Potencial



The Schrödinger equation for the hydrogen atom is a fundamental equation in quantum mechanics that describes the behavior of the electron in the this atom. The potential used in the Schrödinger equation in this case is the Coulomb potential, which is generated by the positively charged nucleus of the hydrogen atom. The Coulomb potential is a classical potential that is proportional to the inverse of the distance between the electron and the nucleus, and it is given by the expression:

\begin{align}
V(t,r,θ,φ) = -\frac{e}{4 \pi ɛ_0 r}
\end{align}

Where $e$ is the charge of the electron, $ε_0$ is the permittivity of free space, and $r$ is the distance between the electron and the nucleus. This potential plays a key role in determining the energy levels and wave functions of the electron in the hydrogen atom, and it is responsible for the formation of the bound states of the atom. The solution of the Schrödinger equation for the hydrogen provides a detailed description of the electronic structure of the simplest atom, and it forms the basis for our understanding of more complex atoms and molecules.


In [135]:
hydrogen = schrodinger.subs(V, -e/(4*sp.pi*ɛ_0*r))
hydrogen

Eq(I*ℏ*Derivative(Ψ(t, r, θ, φ), t), -e*Ψ(t, r, θ, φ)/(4*pi*r*ɛ_0) - ℏ**2*Derivative(r**2*Derivative(Ψ(t, r, θ, φ), r), r)/(2*r**2*μ) - ℏ**2*Derivative(sin(θ)*Derivative(Ψ(t, r, θ, φ), θ), θ)/(2*r**2*μ*sin(θ)) - ℏ**2*Derivative(Ψ(t, r, θ, φ), (φ, 2))/(2*r**2*μ*sin(θ)**2))

### Neural Complex Encoding

One important characteristic of neural networks is that they *only* work with real numbers as input and output data. This means that any complex-valued output, such as a wave function in quantum mechanics, must be separated into its real and imaginary components.

In quantum mechanics, the wave function describes the probability distribution of a particle's position and momentum. It is a complex-valued function, meaning it has both a real and imaginary part. However, neural networks can only work with real numbers, so the wave function must be split into its real and imaginary components. This is done by taking the real part of the wave function and the imaginary part of the wave function, which are then treated as separate outputs to the neural network.

\begin{align}
Ψ(t,r,θ,φ)=Ψ_{re}(t,r,θ,φ)+i Ψ_{im}(t,r,θ,φ)
\end{align}

Where:

\begin{align}
Ψ(t,r,θ,φ): \mathbb{R}^4 →\mathbb{C}
\end{align}

\begin{align}
Ψ_{re}(t,r,θ,φ): \mathbb{R}^4 →\mathbb{R}
\end{align}

\begin{align}
Ψ_{im}(t,r,θ,φ): \mathbb{R}^4 →\mathbb{R}
\end{align}


By splitting the wave function into its real and imaginary parts, the neural network can effectively process the information contained within it. The real and imaginary components can be analyzed separately and combined to generate a complete understanding of the wave function. This approach allows neural networks to solve complex problems in quantum mechanics and other fields that require the analysis of complex-valued data.

Taking the equation of the hydrogen atom and substituting the previous equation:

In [136]:
hydrogen = hydrogen.subs(Ψ,Ψ_re+i*Ψ_im)
hydrogen

Eq(I*ℏ*Derivative(I*Ψ_im(t, r, θ, φ) + Ψ_re(t, r, θ, φ), t), -e*(I*Ψ_im(t, r, θ, φ) + Ψ_re(t, r, θ, φ))/(4*pi*r*ɛ_0) - ℏ**2*Derivative(r**2*Derivative(I*Ψ_im(t, r, θ, φ) + Ψ_re(t, r, θ, φ), r), r)/(2*r**2*μ) - ℏ**2*Derivative(sin(θ)*Derivative(I*Ψ_im(t, r, θ, φ) + Ψ_re(t, r, θ, φ), θ), θ)/(2*r**2*μ*sin(θ)) - ℏ**2*Derivative(I*Ψ_im(t, r, θ, φ) + Ψ_re(t, r, θ, φ), (φ, 2))/(2*r**2*μ*sin(θ)**2))

Let's do the multiplications and other operations:

In [137]:
hydrogen = hydrogen.doit().expand()
hydrogen

Eq(-ℏ*Derivative(Ψ_im(t, r, θ, φ), t) + I*ℏ*Derivative(Ψ_re(t, r, θ, φ), t), -I*e*Ψ_im(t, r, θ, φ)/(4*pi*r*ɛ_0) - e*Ψ_re(t, r, θ, φ)/(4*pi*r*ɛ_0) - I*ℏ**2*Derivative(Ψ_im(t, r, θ, φ), (r, 2))/(2*μ) - ℏ**2*Derivative(Ψ_re(t, r, θ, φ), (r, 2))/(2*μ) - I*ℏ**2*Derivative(Ψ_im(t, r, θ, φ), r)/(r*μ) - ℏ**2*Derivative(Ψ_re(t, r, θ, φ), r)/(r*μ) - I*ℏ**2*Derivative(Ψ_im(t, r, θ, φ), (θ, 2))/(2*r**2*μ) - ℏ**2*Derivative(Ψ_re(t, r, θ, φ), (θ, 2))/(2*r**2*μ) - I*ℏ**2*cos(θ)*Derivative(Ψ_im(t, r, θ, φ), θ)/(2*r**2*μ*sin(θ)) - ℏ**2*cos(θ)*Derivative(Ψ_re(t, r, θ, φ), θ)/(2*r**2*μ*sin(θ)) - I*ℏ**2*Derivative(Ψ_im(t, r, θ, φ), (φ, 2))/(2*r**2*μ*sin(θ)**2) - ℏ**2*Derivative(Ψ_re(t, r, θ, φ), (φ, 2))/(2*r**2*μ*sin(θ)**2))

Let's now separate both real part and imaginary parts of this equation. For this, let's write a function that receives the equation of format:

\begin{align}
A+Bi = C+Di
\end{align}

And returns:

\begin{align}
A - C = 0
\end{align}

\begin{align}
B - D = 0
\end{align}

In [138]:
def split_complex(equation):

    """

    Returns a tuple of two equations representing the real and imaginary parts
    of the input equation, which is assumed to involve complex numbers.

    Parameters
    ----------
    equation : sympy.core.relational.Equality
        The equation to extract the real and imaginary parts from.

    Returns
    -------
    (real_equation, imag_equation): tuple of sympy.core.relational.Equality
        A tuple containing two equations representing the real and imaginary
        parts of the input equation, respectively.

    """
    
    # Extract the real and imaginary parts of the left-hand side minus the right-hand side
    real_equation = sp.re(equation.lhs - equation.rhs)
    imag_equation = sp.im(equation.lhs - equation.rhs)
    
    # Replace any real part of a derivative that has an imaginary expression with the real part
    real_equation = real_equation.replace(lambda x: isinstance(x, sp.re) and
                                           x.args[0].is_Derivative and
                                           x.args[0].expr.is_real,
                                           lambda x: x.args[0])
    
    imag_equation = imag_equation.replace(lambda x: isinstance(x, sp.re) and
                                           x.args[0].is_Derivative and
                                           x.args[0].expr.is_real,
                                           lambda x: x.args[0])
    
    # Replace any real part of a derivative that does not have an imaginary expression with 0
    real_equation = real_equation.replace(lambda x: isinstance(x, sp.im) and
                                           x.args[0].is_Derivative and
                                           x.args[0].expr.is_real,
                                           lambda x: 0)
    
    imag_equation = imag_equation.replace(lambda x: isinstance(x, sp.im) and
                                           x.args[0].is_Derivative and
                                           x.args[0].expr.is_real,
                                           lambda x: 0)
    
    # Create new equations by setting the real and imaginary parts to 0
    real_equation = sp.Eq(real_equation, 0)
    imag_equation = sp.Eq(imag_equation, 0)
    
    # Return a tuple containing the real and imaginary part equations, respectively
    return (real_equation, imag_equation)

In [139]:
real_hydrogen_equation, imag_hydrogen_equation = split_complex(hydrogen)

So, our neural network must satisfy the equations:

In [140]:
real_hydrogen_equation

Eq(e*Ψ_re(t, r, θ, φ)/(4*pi*r*ɛ_0) - ℏ*Derivative(Ψ_im(t, r, θ, φ), t) + ℏ**2*Derivative(Ψ_re(t, r, θ, φ), (r, 2))/(2*μ) + ℏ**2*Derivative(Ψ_re(t, r, θ, φ), r)/(r*μ) + ℏ**2*Derivative(Ψ_re(t, r, θ, φ), (θ, 2))/(2*r**2*μ) + ℏ**2*cos(θ)*Derivative(Ψ_re(t, r, θ, φ), θ)/(2*r**2*μ*sin(θ)) + ℏ**2*Derivative(Ψ_re(t, r, θ, φ), (φ, 2))/(2*r**2*μ*sin(θ)**2), 0)

In [141]:
imag_hydrogen_equation

Eq(e*Ψ_im(t, r, θ, φ)/(4*pi*r*ɛ_0) + ℏ*Derivative(Ψ_re(t, r, θ, φ), t) + ℏ**2*Derivative(Ψ_im(t, r, θ, φ), (r, 2))/(2*μ) + ℏ**2*Derivative(Ψ_im(t, r, θ, φ), r)/(r*μ) + ℏ**2*Derivative(Ψ_im(t, r, θ, φ), (θ, 2))/(2*r**2*μ) + ℏ**2*cos(θ)*Derivative(Ψ_im(t, r, θ, φ), θ)/(2*r**2*μ*sin(θ)) + ℏ**2*Derivative(Ψ_im(t, r, θ, φ), (φ, 2))/(2*r**2*μ*sin(θ)**2), 0)

### Hartree Atomic Units

The Hartree atomic units system is a natural system of units used in quantum mechanics that simplifies the mathematical expressions involved in solving the Schrödinger equation for atoms and molecules. By setting fundamental physical constants to unity, the equations become simpler and easier to solve numerically, which is particularly useful in numerical quantum mechanics. The use of Hartree atomic units provides a more intuitive understanding of the physics involved in the behavior of atoms and molecules, and is widely used in theoretical and computational studies in the field of quantum mechanics. It has become an essential tool in the development of new materials and technologies. For more information, [Hartree Atomic Units](https://en.wikipedia.org/wiki/Hartree_atomic_units#:~:text=The%20Hartree%20atomic%20units%20are,after%20the%20physicist%20Douglas%20Hartree.).

In this system, we have:
\begin{align}
  ℏ = 1 \quad \text{(Reduced Planck Constant)}
\end{align}
\begin{align}
  e = 1 \quad \text{(Elementary Charge)}
\end{align}
\begin{align}
  m_e = 1 \quad \text{(Electron Rest Mass)}
\end{align}
\begin{align}
  a_0 = 1 \quad \text{(Bohr Radius)}
\end{align}

As consequences of these definitions, we also have:

\begin{align}
   ɛ_0 = \frac{1}{4 π} \quad \text{(Permittivity of Free Space)}
\end{align}

Considering that the mass of a proton satisfies:

\begin{align}
m_p ≈ 1836.1527 m_e
\end{align}

With this, we can find the reduced mass of the system $\mu$:

\begin{align}
μ = \frac{m_p \cdot m_e}{m_p + m_e} \approx 0.999 \cdot m_e \approx 1\cdot m_e 
\end{align}

With this approximation, we can considerer:

\begin{align}
μ \approx 1
\end{align}

Doing the substitution:

In [150]:
real_hydrogen_equation = real_hydrogen_equation.subs({e:1,ℏ:1,μ:1,ɛ_0:1/(4*sp.pi)})
real_hydrogen_equation

Eq(-Derivative(Ψ_im(t, r, θ, φ), t) + Derivative(Ψ_re(t, r, θ, φ), (r, 2))/2 + Ψ_re(t, r, θ, φ)/r + Derivative(Ψ_re(t, r, θ, φ), r)/r + Derivative(Ψ_re(t, r, θ, φ), (θ, 2))/(2*r**2) + cos(θ)*Derivative(Ψ_re(t, r, θ, φ), θ)/(2*r**2*sin(θ)) + Derivative(Ψ_re(t, r, θ, φ), (φ, 2))/(2*r**2*sin(θ)**2), 0)

In [151]:
imag_hydrogen_equation = imag_hydrogen_equation.subs({e:1,ℏ:1,μ:1,ɛ_0:1/(4*sp.pi)})
imag_hydrogen_equation

Eq(Derivative(Ψ_im(t, r, θ, φ), (r, 2))/2 + Derivative(Ψ_re(t, r, θ, φ), t) + Ψ_im(t, r, θ, φ)/r + Derivative(Ψ_im(t, r, θ, φ), r)/r + Derivative(Ψ_im(t, r, θ, φ), (θ, 2))/(2*r**2) + cos(θ)*Derivative(Ψ_im(t, r, θ, φ), θ)/(2*r**2*sin(θ)) + Derivative(Ψ_im(t, r, θ, φ), (φ, 2))/(2*r**2*sin(θ)**2), 0)

### Avoiding mathematical indeterminacies in the equations

To avoid mathematical difficulties and errors when dealing with equations that involve division by radial or angular factors in quantum mechanics, a common technique is to multiply both sides of the equation by the reciprocal of the factor, which cancels out the denominator and simplifies the expression. This technique is widely used in the study of atoms and molecules, where the Schrödinger equation involves terms that are proportional to $r^2$ and $sin^2(θ)$. By applying this approach, numerical calculations are made more accurate and reliable, thus ensuring the validity of results.

Then, let's multiply the real equation by the term $2 r^2 sin^2(θ)$:

In [157]:
real_hydrogen_equation = sp.Eq((real_hydrogen_equation.lhs*(2*(r**2)*(sp.sin(θ)**2))).expand(),0)
real_hydrogen_equation

Eq(-2*r**2*sin(θ)**2*Derivative(Ψ_im(t, r, θ, φ), t) + r**2*sin(θ)**2*Derivative(Ψ_re(t, r, θ, φ), (r, 2)) + 2*r*Ψ_re(t, r, θ, φ)*sin(θ)**2 + 2*r*sin(θ)**2*Derivative(Ψ_re(t, r, θ, φ), r) + sin(θ)**2*Derivative(Ψ_re(t, r, θ, φ), (θ, 2)) + sin(θ)*cos(θ)*Derivative(Ψ_re(t, r, θ, φ), θ) + Derivative(Ψ_re(t, r, θ, φ), (φ, 2)), 0)

Doing the same multiplication with the imaginary equation:

In [158]:
imag_hydrogen_equation = sp.Eq((imag_hydrogen_equation.lhs*(2*(r**2)*(sp.sin(θ)**2))).expand(),0)
imag_hydrogen_equation

Eq(r**2*sin(θ)**2*Derivative(Ψ_im(t, r, θ, φ), (r, 2)) + 2*r**2*sin(θ)**2*Derivative(Ψ_re(t, r, θ, φ), t) + 2*r*Ψ_im(t, r, θ, φ)*sin(θ)**2 + 2*r*sin(θ)**2*Derivative(Ψ_im(t, r, θ, φ), r) + sin(θ)**2*Derivative(Ψ_im(t, r, θ, φ), (θ, 2)) + sin(θ)*cos(θ)*Derivative(Ψ_im(t, r, θ, φ), θ) + Derivative(Ψ_im(t, r, θ, φ), (φ, 2)), 0)

### Final Equations

Through the process of equation engineering, we were able to preprocess the equation to the neural network. By simplifying and manipulating mathematical expressions, we uncovered relationships and patterns that revealed the appropriate differential equations necessary for accurate modeling and analysis. These differential equations allowed us to make predictions and explore the behavior of systems over time:

* Real Equation:

In [166]:
real_hydrogen_equation

Eq(-2*r**2*sin(θ)**2*Derivative(Ψ_im(t, r, θ, φ), t) + r**2*sin(θ)**2*Derivative(Ψ_re(t, r, θ, φ), (r, 2)) + 2*r*Ψ_re(t, r, θ, φ)*sin(θ)**2 + 2*r*sin(θ)**2*Derivative(Ψ_re(t, r, θ, φ), r) + sin(θ)**2*Derivative(Ψ_re(t, r, θ, φ), (θ, 2)) + sin(θ)*cos(θ)*Derivative(Ψ_re(t, r, θ, φ), θ) + Derivative(Ψ_re(t, r, θ, φ), (φ, 2)), 0)

* Imaginary Equation:

In [165]:
imag_hydrogen_equation

Eq(r**2*sin(θ)**2*Derivative(Ψ_im(t, r, θ, φ), (r, 2)) + 2*r**2*sin(θ)**2*Derivative(Ψ_re(t, r, θ, φ), t) + 2*r*Ψ_im(t, r, θ, φ)*sin(θ)**2 + 2*r*sin(θ)**2*Derivative(Ψ_im(t, r, θ, φ), r) + sin(θ)**2*Derivative(Ψ_im(t, r, θ, φ), (θ, 2)) + sin(θ)*cos(θ)*Derivative(Ψ_im(t, r, θ, φ), θ) + Derivative(Ψ_im(t, r, θ, φ), (φ, 2)), 0)